In [1]:
%load_ext kedro

[10/09/25 19:21:57] INFO     Using                                                                  ]8;id=323896;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=754462;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-pa                
                             ckages/kedro/framework/project/rich_logging.yml' as logging                           
                             configuration.                                                                        

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=302234;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=158158;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=725292;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=494058;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro/ipython/__init__.py#65\65]8;;\

                    INFO     Resolved project path as:                                              ]8;id=524957;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=340250;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro/ipython/__init__.py#180\180]8;;\
                             /home/zishan/Documents/projects/imdb-review-sentiment-analysis.                       
                             To set a different path, run '%reload_kedro <project_root>'                           

[10/09/25 19:21:58] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=438352;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=419670;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro_telemetry/plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[10/09/25 19:21:59] INFO     Kedro project imdb_review_sentiment_analysis                           ]8;id=590072;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=139240;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro/ipython/__init__.py#146\146]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=749036;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=568817;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro/ipython/__init__.py#147\147]8;;\
                             'pipelines'                                                                           

In [2]:
df = catalog.load('clean_data')
df.info()

[10/09/25 19:22:04] INFO     Loading data from clean_data (ParquetDataset)...                  ]8;id=907960;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=782852;file:///home/zishan/anaconda3/envs/sentiment_analysis/lib/python3.11/site-packages/kedro/io/data_catalog.py#1046\1046]8;;\

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   processed_text  25000 non-null  object
 1   pos_senti       25000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 390.8+ KB


In [8]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import gensim
from gensim.models import Word2Vec

In [4]:
cnt_vect = CountVectorizer(max_features=5000, min_df=2, max_df=0.95)
cnt_mat = cnt_vect.fit_transform(df['processed_text'])

X = cnt_mat
y = df['pos_senti']

In [5]:
tfidf_vect = TfidfVectorizer(max_features=5000, min_df=2, max_df=0.95)
tfidf_mat = tfidf_vect.fit_transform(df['processed_text'])

X = tfidf_mat
y = df['pos_senti']

In [7]:
sentences = [text.split() for text in df['processed_text']]

w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4
)

def get_document_vector(words, model):
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

X_w2v = np.array([get_document_vector(words, w2v_model) for words in sentences])
y = df['pos_senti']